In [ ]:
# open and read Cassie's data
import random
from matplotlib import pyplot as plt
import numpy as np
import math
from sklearn.svm import SVC
#from sklearn.datasets import make_classification
from sklearn.model_selection import GridSearchCV
import warnings
import pandas as pd
from sklearn.preprocessing import MinMaxScaler as Scaler
scaler = Scaler()
from sklearn.model_selection import train_test_split
from sklearn import metrics
warnings.simplefilter(action='ignore', category=Warning)
from google.colab import drive
drive.mount('/content/gdrive')
inputfile = "/content/gdrive/My Drive/Landslide Files/Copy of ls_data_SVM.1_28_2021.csv" 

### dataset specifies any feature to be dropped from the anaylsis 
dataset = 'TEST' # 
# log10(surface.area)

with open(inputfile, 'r') as f:
  data = f.read()
  print(repr(data)[:100])
  print(repr(data)[-100:])
  print(len(data))
f.close()
lines = data.split('\n')
lines.pop()
print(len(lines))
print(lines[0])
print(lines[1])

headings = lines[0].split(',')
print(headings)

table =[]
for x in range(1, len(lines)):
  row = lines[x].split(',')
  if row[len(row)-1] != 'NA':
    for y in range(1, len(row)): row[y] = float(row[y])
    table.append([ row[0], row[15], row[7], row[8], row[9], row[3], math.log10(row[16]),row[5], row[17]/row[16], row[20] ])
""" 
landslides_polygons$id,mean.tch,sd.tch,mean.slope,mean.tri,tri.10m,elev,elev.km,ls.northing,ls.easting,insol.dsm,insol.kw,insol,agb,year,age,surface.area,perimeter,elev.mean.tch,max.tch.buf,rem
  0                     1        2      3          4        5      6     7       8           9          10        11       12   13  14   15   16          17         18           19          20
ID, age, elev.km, ls.northing, ls.easting, mean.slope, log10(surface.area), tri.10m, pa.ratio, rem 
0   1    2        3            4           5           6                    8        9         10       
"""
### make young and old datasets.  The young data will be split into train&test for SVC, then that model will be applied to old_df, and result placed in oldraw
youngraw, oldraw =[], [] 
for x in range(len(table)):
  if table[x][1] <= 5:
    youngraw.append(table[x])
  else: 
    table[x].append(0)
    oldraw.append(table[x])
### make young_df, which will be split and used for SVC below
new_headings = [ 'ID', 'age', 'elev.km', 'ls.northing', 'ls.easting', 'mean.slope', 'log10(surface.area)',  'tri.10m', 'pa.ratio', 'rem']
young_df = pd.DataFrame(youngraw, columns=new_headings)

### make old_df and preprocess it.  It will be ready for predictions
old_headings = [ 'ID', 'age', 'elev.km', 'ls.northing', 'ls.easting', 'mean.slope', 'log10(surface.area)',  'tri.10m', 'pa.ratio', 'old.rem', 'svmrem' ]
old_df = pd.DataFrame(oldraw, columns = old_headings)
old_df_labels = old_df["old.rem"].copy()
old_df = old_df.drop("old.rem", axis=1)
old_df = old_df.drop("svmrem", axis=1)
old_df = old_df.drop("age", axis=1)
old_df = old_df.drop("ID", axis=1)
#old_df = old_df.drop('log10(surface.area)', axis=1)
#old_df = old_df.drop('ls.easting', axis=1)
#old_df = old_df.drop('ls.northing', axis=1)

scaler.fit(old_df) 
old_df_scaled = scaler.transform(old_df)

### oldraw has all the data for old sites, old_df_scaled is ready for predictions

In [ ]:
# pandas treatments and SVM routines

from sklearn.model_selection import train_test_split
import statistics as stats

scorefile = "/content/gdrive/My Drive/ML Files/Accuracy.Precision.Recall." +dataset+ ".csv" 
predictionfile =  "/content/gdrive/My Drive/ML Files/Predictions." +dataset+ ".csv"
start =0
finish = 100 
if start ==0:
  with open(scorefile, 'w') as e: 
    e.close()
  with open(predictionfile, 'w') as d: 
    for x in range(len(oldraw)): 
      d.write('0,')
  d.close()

accuracies, recall, precision =[], [], []
for a in range(start, finish):  # for drop: 'log(surface.area),pa.ratio', we have 0-2 and 86-99.  Need 3-86
  #if a% 10 ==0: print('Working on', a)
  train_set, test_set = train_test_split(young_df, test_size=0.5, random_state = a )  
### assign rem as value to be predicted, drop columns to be excluded 
  train_set_labels = train_set["rem"].copy()
  train_set = train_set.drop("rem", axis=1)
  train_set = train_set.drop("age", axis=1)
  train_set = train_set.drop("ID", axis=1)
  #train_set = train_set.drop("log10(surface.area)", axis=1)
  #train_set = train_set.drop('ls.easting', axis =1)
  #train_set = train_set.drop('ls.northing', axis =1)

  test_set_labels = test_set["rem"].copy()
  test_set = test_set.drop("rem", axis=1)
  test_set = test_set.drop("age", axis=1)
  test_set = test_set.drop("ID", axis=1)
  #test_set = test_set.drop("log10(surface.area)", axis=1)
  #test_set = test_set.drop('ls.easting', axis =1)
  #test_set = test_set.drop('ls.northing', axis =1)

### the SVM routines
  scaler.fit(train_set) 
  train_set_scaled = scaler.transform(train_set)
  test_set_scaled = scaler.transform(test_set)
  clf = SVC()  # SVC, NuSVC
  param_grid = {
        'C': [0.1, 1.0, 10.0, 50.0], 
        'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
        'shrinking': [True, False],
        'degree': [1,2,3,4,5,6], 
        'gamma': ['auto', 'scale', 1, 0.1, 0.01],
        'coef0': [0.0, 0.1, 0.5]  }

  grid_search = GridSearchCV(
          clf, param_grid, cv=10, scoring='accuracy')
  grid_search.fit(train_set_scaled, train_set_labels)
  X = np.append(train_set_scaled, test_set_scaled, axis=0)
  y = np.append(train_set_labels, test_set_labels, axis=0)
  CLF = clf.fit(X, y)
### prediction statistics on test set
  predicted_y = clf.predict(test_set_scaled)
  RECALL = metrics.recall_score(test_set_labels, predicted_y)
  PRECISION = metrics.precision_score(test_set_labels, predicted_y)
  ACCURACY = metrics.accuracy_score(test_set_labels, predicted_y)
  print(a, grid_search.best_score_, ACCURACY, PRECISION, RECALL)
  accuracies.append(ACCURACY)
  recall.append(RECALL)
  precision.append(PRECISION)
  with open(scorefile, 'a') as g: g.write(str(a) +','+ str(ACCURACY) +',' +str(PRECISION)+ ',' + str(RECALL) +'\n') 
  g.close()
### predictions for old sites
  prediction = clf.predict(old_df_scaled)
  with open(predictionfile, 'r') as e:
    predraw = e.read()
  e.close()
  preds = predraw.split(',')
  preds.pop()
  for x in range(len(prediction)):
    if prediction[x] == 1: 
      preds[x] = str(int(preds[x]) +1) 
  with open(predictionfile, 'w') as e:
    for y in range(len(preds)): e.write(preds[y] + ',')
  e.close()
print('Dataset=', dataset, 'Mean accuracy =', stats.mean(accuracies), ', sem =', stats.stdev(accuracies) / math.sqrt(len(accuracies) ) )
print('Done.')
